In [1]:
from bs4 import BeautifulSoup
import requests
import urllib, json
import re
import pandas as pd

In [2]:
def Link_extract(category):
    category_url='https://ekantipur.com/'+category+''
    r=requests.get(category_url)
    sporthtml=r.text
    soup=BeautifulSoup(sporthtml,'html.parser')
    label=soup.findAll("div", {"class": "catName"})
    links=[]
    for i in label:
        for j in i:
            label=j
    for div in soup.findAll("div", {"class": "teaser offset"}):
        a = div.findAll('a')[0]
        link=a.attrs['href']
        links.append("https://ekantipur.com"+link)
    return links,label


In [3]:
def ajax_link(category):
    ajax_link=[]
    links,_=Link_extract(category)
    for i in range(1,29,2):
        my_url = 'https://ekantipur.com/'+category+'/2019/10/'+str(i)+'?json=true' 
        ret = requests.get(my_url)
        response = urllib.request.urlopen(my_url)
        data = json.loads(response.read())
        data=data['html']
        urls = re.findall('(?:(?:https?|ftp)://ekantipur.com/'+category+'/)[\w/\-?=%.]+\.[\w/\-?=%.]+', data)

        for i in urls:
            ajax_link.append(i)
    for s in list(set(ajax_link)):
        links.append(s)
    return links


In [4]:
def Get_data(category):
    final_data=[]
    data=[]
    links=ajax_link(category)
    for i in range(len(links)):
        soup = BeautifulSoup(requests.get(links[i]).text,"html.parser")
        for item in soup.findAll("div", class_=['description', 'portrait']):
            if item['class'][0]=='description':
                required_data = [p_item.text.strip() for p_item in item.select("p")]
                required_data=required_data[0:2]
                for a in required_data:
                    data.append(a)      
        final_data.append(" ".join(data[0:2]))
        data=[]
    return final_data


In [5]:
sport_data=Get_data('sports')
enentertainment_data=Get_data('entertainment')
business_data=Get_data('business')
opinion_data=Get_data('opinion')

In [6]:
def Get_label(category,data):
    label=[]
    lbl=Link_extract(category)[1]
    for i in range(len(data)):
        label.append(lbl)
    return label

In [7]:
sport_label=Get_label('sports',sport_data)
enentertainment_label=Get_label('entertainment',enentertainment_data)
business_label=Get_label('business',business_data)
opinion_label=Get_label('opinion',opinion_data)

In [8]:
sport={'news':sport_data,
  'label':sport_label}
enentertainment={'news':enentertainment_data,
  'label':enentertainment_label}
business={'news':business_data,
  'label':business_label}
opinion={'news':opinion_data,
  'label':opinion_label}

In [9]:
df1=pd.DataFrame(sport)
df2=pd.DataFrame(enentertainment)
df3=pd.DataFrame(business)
df4=pd.DataFrame(opinion)

In [10]:
scarping_data=pd.concat([df1,df2,df3,df4])

In [2]:
scarping_data.to_csv('scarping_news.csv')

NameError: name 'scarping_data' is not defined